In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
import json
from openai import OpenAI

# Applied LLM Systems - Lab Exercise 1: City-Country Mapping

## 🎯 Objective

Build an LLM-based system that converts a text file of city names into a structured JSON file mapping each city to its country.

## 📋 Task Description

You are given a text file (`cities.txt`) containing city names. Your task is to create a system that outputs a JSON file (`cities_countries.json`) with this structure:

```json
{
  "Tokyo": "Japan",
  "Paris": "France",
  "New York": "United States",
  "Sydney": "Australia"
}
```

## 🔧 Requirements

1. Use any LLM provider of your choice (OpenAI, Anthropic, Cohere, etc.)
2. Acquire an API key from your chosen provider's platform
3. Store the API key as an environment variable
4. Write code that:
   - Reads the input text file
   - Processes city names using the LLM
   - Outputs valid JSON in the required format




In [3]:
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise RuntimeError("Set OPENAI_API_KEY")

client = OpenAI(api_key=openai_api_key)

with open(fr"assets\task1\in\cities.txt", "r", encoding="utf-8") as f:
    cities = [line.strip() for line in f if line.strip()]

user_prompt = (
    "You are a careful assistant that returns strict JSON only."
    "Map each given city name to its country. Use English country names."
    "Do not add extra keys. Do not include comments or explanations."
    "If a city is ambiguous, or has syntax or, grammar errors, extra spaces, or extra or missing letters, pick the best known match."
    "If the city is not real, use 'Unknown' as the country value."
    "Return a JSON object that maps each city to its country."
    "Use exactly the provided city strings as keys. Cities:\n\n" + "\n".join(cities)
)

resp = client.chat.completions.create(
    model=os.getenv("MODEL", "gpt-4o-mini"),
    messages=[
        {"role": "user", "content": user_prompt}
    ],
    response_format={"type": "json_object"},
    temperature=0,
)

data = json.loads(resp.choices[0].message.content)

out = {city: str(data.get(city, "")) for city in cities}

with open(fr"assets\task1\out\cities_countries.json", "w", encoding="utf-8") as f:
    json.dump(out, f, indent=2, ensure_ascii=False)

print("Saved cities_countries.json")

Saved cities_countries.json


# Applied LLM Systems - Lab Exercise 2: Finding Closest Cities

## 🎯 Objective

Use LLMs as tools to solve a multi-step problem: finding the pair of cities with the shortest distance between them. This exercise demonstrates how to break down complex tasks and use LLMs to generate both data and code.

## 📋 Task Description

Given your list of cities from the previous exercise, find which two cities are geographically closest to each other.

**Important**: LLMs typically cannot directly calculate distances between cities accurately. Instead, you'll use the LLM as a tool-generating assistant.

## 🔧 Your Workflow

### Step 1: Generate Coordinates
Ask your LLM to generate geographical coordinates (latitude and longitude) for each city from your list. Save this as a JSON file:

```json
{
  "Tokyo": {"lat": 35.6762, "lon": 139.6503},
  "Paris": {"lat": 48.8566, "lon": 2.3522},
  "Berlin": {"lat": 52.5200, "lon": 13.4050}
}
```

### Step 2: Generate Distance Calculator
Ask your LLM to write a Python script that:
- Reads the coordinates JSON file
- Calculates distances between all pairs of cities (brute force is fine)
- Identifies and prints the closest pair

Save this generated code as `main.py`

### Step 3: Execute
Run the generated script (from notebook) with your coordinates JSON:
```bash
python main.py --json_path cordinates.json
```

The output should identify the closest pair and their distance.




In [4]:
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise RuntimeError("Set OPENAI_API_KEY")

client = OpenAI(api_key=openai_api_key)

with open(fr"assets\task2\in\cities.txt", "r", encoding="utf-8") as f:
    cities = [line.strip() for line in f if line.strip()]

system_prompt = (
    "You are a careful assistant that returns strict JSON only. "
    "Given a list of city names, produce a JSON object where each key is the exact city string, "
    "and each value is an object with numeric fields 'lat' and 'lon' in decimal degrees. "
    "Use English place names only. "
    "If a city string is ambiguous or slightly misspelled, choose the best known match. "
    "If the city does not exist, return {\"lat\": null, \"lon\": null} for that city. "
    "Do not include comments or explanations."
)

user_prompt = "Cities:\n\n" + "\n".join(cities)

resp = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
    response_format={"type": "json_object"},
    temperature=0,
)

coords = json.loads(resp.choices[0].message.content)

out = {}

for c in cities:
    v = coords.get(c, {})
    lat = v.get("lat", None)
    lon = v.get("lon", None)

    try:
        lat = float(lat) if lat is not None else None
    except Exception:
        lat = None
    try:
        lon = float(lon) if lon is not None else None
    except Exception:
        lon = None

    out[c] = {"lat": lat, "lon": lon}

with open(fr"assets\task2\out\city_coords.json", "w", encoding="utf-8") as f:
    json.dump(out, f, indent=2, ensure_ascii=False)

print("Saved city_coords.json")

Saved city_coords.json


In [5]:
prompt = (
    "Write a complete Python 3 script named main.py. "
    "Do not include backticks. Do not add explanations. "
    "The script must:\n"
    "1) Accept a command line argument --json_path that points to a JSON file with structure:\n"
    "{\n"
    '  "CityA": {"lat": 12.34, "lon": 56.78},\n'
    '  "CityB": {"lat": 90.12, "lon": 34.56}\n'
    "}\n"
    "2) Load the file, ignore entries where lat or lon is null. "
    "3) Compute great circle distances in kilometers for all city pairs using the Haversine formula. "
    "4) Find the closest pair. "
    "5) Print exactly these lines:\n"
    "Closest pair: <City1> and <City2>\n"
    "Distance (km): <number with 3 decimals>\n"
    "6) Use only Python standard library. "
    "7) Handle empty or single city gracefully with a clear message and exit code 0."
)

resp = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    temperature=0,
)

code = resp.choices[0].message.content

with open(fr"assets\task2\out\main.py", "w", encoding="utf-8") as f:
    f.write(code)

print("Wrote main.py")

Wrote main.py


In [6]:
!python assets\task2\out\main.py --json_path assets/task2/out/city_coords.json


Closest pair: Paris and madrid
Distance (km): 1052.896
